<a href="https://colab.research.google.com/github/Nagendrajay/Lecture-notes/blob/main/445_Network_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Analysis

Many interesting data problems can be fruitfully thought of in terms of **networks**, consisting of `nodes` of some type and the `edges` that join them.

For instance, your Facebook friends form the `nodes` of a network whose `edges` are friendship relations. A less obvious example is the World Wide Web itself, with each web page a `node` and each hyperlink from one page to another an `edge`. The nodes can be `undirected` (connecting two nodes both ways) or `directed` (connecting two nodes one way only).

## Metrics

1. **degree centrality** - most connected nodes.
2. **betweenness centrality** - identifies nodel which frequently are on the shortest paths between pairs of other nodel. In particular, the betweenness centrality of node i is computed by adding up, for every other pair of nodes j and k, the proportion of shortest paths between node j and node k that pass through i.
3. **closesess centrality** - XXXXXXXXXXXXXXX
4. **eigenvector centrality** - nodes with high eigenvector centrality should be those who have a lot of connections, and connections to other nodes whis themselves have high centrality.
5. **number of endorsements** - or any other user-based activities.
6. **PageRank**

In [ ]:
users = [
        { "id": 0, "name": "Hero" },
        { "id": 1, "name": "Dunn" },
        { "id": 2, "name": "Sue" },
        { "id": 3, "name": "Chi" },
        { "id": 4, "name": "Thor" },
        { "id": 5, "name": "Clive" },
        { "id": 6, "name": "Hicks" },
        { "id": 7, "name": "Devin" },
        { "id": 8, "name": "Kate" },
        { "id": 9, "name": "Klein" }
]

In [ ]:
friendship_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
                   (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

For example, the tuple (0, 1) indicates that the data scientist with id 0 (Hero) and the data scientist with id 1 (Dunn) are friends.

<img src="images/fig01-01.png" alt="Analytics, Data Science, Machine Learning" style="width: 800px;"/>


## 1. Degree centrality - Finding Key Connectors

In [ ]:
# Find friends
def find_friends(node_id, friendship_pairs):
    '''
    Returns friends of a specific person using friendship_pairs.

    Args:
        node_id (int): The id of a node.
        friendship_map (list): a list of (X, Y) tuples, where X and Y are nodes id.

    Returns:
        friends (list): a list of friends id.
    '''
    friends = []
    for friendship in friendship_pairs:
        if friendship[0] == node_id:
            friends.append(friendship[1])
        if friendship[1] == node_id:
            friends.append(friendship[0])
    return friends

In [ ]:
find_friends(1, friendship_pairs)

[0, 2, 3]

In [ ]:
# Initialize the dict with an empty list for each user id:
friendships = {user["id"]: [] for user in users}
friendships

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}

In [ ]:
# And loop over the friendship pairs to populate it:
for i, j in friendship_pairs:
    friendships[i].append(j)  # Add j as a friend of user i
    friendships[j].append(i)  # Add i as a friend of user j

In [ ]:
friendships

{0: [1, 2],
 1: [0, 2, 3],
 2: [0, 1, 3],
 3: [1, 2, 4],
 4: [3, 5],
 5: [4, 6, 7],
 6: [5, 8],
 7: [5, 8],
 8: [6, 7, 9],
 9: [8]}

### What's the average number of connections?

In [ ]:
def number_of_friends(user, friendships):
    '''
    Returns a number friends of a specific user and friendships.

    Args:
        user (list): a list of dics { "id": 0, "name": "Hero" }
        friendships (dict): a list of dics {0: [1, 2]}

    Returns:
        friends_count (int): number of friends of a specified user.
    '''
    user_id = user["id"]
    #print(user_id)
    friend_ids = friendships[user_id]
    #print(friend_ids)
    friends_count = len(friend_ids)
    #print(friends_count)
    return friends_count

In [ ]:
number_of_friends(users[1], friendships)

3

In [ ]:
total_connections = sum(number_of_friends(user, friendships) for user in users)
total_connections

24

In [ ]:
num_users = len(users)
num_users

10

In [ ]:
avg_connections = total_connections / num_users
avg_connections

2.4

### Who are the most connected people?

In [ ]:
# Create a list (user_id, number_of_friends)
num_friends_by_id = [(user["id"], number_of_friends(user, friendships)) for user in users]
num_friends_by_id

[(0, 2),
 (1, 3),
 (2, 3),
 (3, 3),
 (4, 2),
 (5, 3),
 (6, 2),
 (7, 2),
 (8, 3),
 (9, 1)]

In [ ]:
# Compute the network metric degree centrality
num_friends_by_id.sort(                                # Sort the list
       key=lambda id_and_friends: id_and_friends[1],   # by num_friends
       reverse=True)                                   # largest to smallest

# Each pair is (user_id, num_friends):
# [(1, 3), (2, 3), (3, 3), (5, 3), (8, 3),
#  (0, 2), (4, 2), (6, 2), (7, 2), (9, 1)]
num_friends_by_id

[(1, 3),
 (2, 3),
 (3, 3),
 (5, 3),
 (8, 3),
 (0, 2),
 (4, 2),
 (6, 2),
 (7, 2),
 (9, 1)]

<img src="images/fig01-02.png" alt="Analytics, Data Science, Machine Learning" style="width: 800px;"/>


## 2. Betweenness centrality

Betweenness centrality is a measure of a node's importance within a network based on its position between other nodes. In simpler terms, it quantifies how often a node lies on the shortest path between any pair of other nodes in the network. In NLP, we often represent text as graphs, where nodes are words or concepts, and edges represent relationships between them. Betweenness centrality can be used to:

•	Identify key words or phrases: Words with high betweenness centrality are often central to the text's meaning.
•	Extract summaries: By focusing on nodes with high betweenness centrality, we can generate concise summaries.
•	Analyze sentence structure: Understanding the role of words in a sentence based on their position in the dependency graph.
•	Information retrieval: Identifying important documents in a collection based on their connections to other documents.
Limitations
•	Computational cost: Calculating betweenness centrality can be computationally expensive for large graphs.
•	Sensitivity to noise: It can be sensitive to noise in the graph structure.
•	Doesn't capture all aspects of importance: While it measures one type of centrality, it may not capture other important features of nodes.


In [ ]:
from typing import NamedTuple

class User(NamedTuple):
    id: int
    name: str

users = [User(0, "Hero"), User(1, "Dunn"), User(2, "Sue"), User(3, "Chi"),
         User(4, "Thor"), User(5, "Clive"), User(6, "Hicks"),
         User(7, "Devin"), User(8, "Kate"), User(9, "Klein")]

In [ ]:
friend_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
                (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [ ]:
from typing import Dict, List

# type alias for keeping track of Friendships
Friendships = Dict[int, List[int]]

friendships: Friendships = {user.id: [] for user in users}

for i, j in friend_pairs:
    friendships[i].append(j)
    friendships[j].append(i)

assert friendships[4] == [3, 5]
assert friendships[8] == [6, 7, 9]

In [ ]:
from collections import deque

Path = List[int]

def shortest_paths_from(from_user_id: int,
                        friendships: Friendships) -> Dict[int, List[Path]]:
    # A dictionary from user_id to *all* shortest paths to that user.
    shortest_paths_to: Dict[int, List[Path]] = {from_user_id: [[]]}

    # A queue of (previous user, next user) that we need to check.
    # Starts out with all pairs (from_user, friend_of_from_user).
    frontier = deque((from_user_id, friend_id)
                     for friend_id in friendships[from_user_id])

    # Keep going until we empty the queue.
    while frontier:
        # Remove the pair that's next in the queue.
        prev_user_id, user_id = frontier.popleft()

        # Because of the way we're adding to the queue,
        # necessarily we already know some shortest paths to prev_user.
        paths_to_prev_user = shortest_paths_to[prev_user_id]
        new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]

        # It's possible we already know a shortest path to user_id.
        old_paths_to_user = shortest_paths_to.get(user_id, [])

        # What's the shortest path to here that we've seen so far?
        if old_paths_to_user:
            min_path_length = len(old_paths_to_user[0])
        else:
            min_path_length = float('inf')

        # Only keep paths that aren't too long and are actually new.
        new_paths_to_user = [path
                             for path in new_paths_to_user
                             if len(path) <= min_path_length
                             and path not in old_paths_to_user]

        shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user

        # Add never-seen neighbors to the frontier.
        frontier.extend((user_id, friend_id)
                        for friend_id in friendships[user_id]
                        if friend_id not in shortest_paths_to)

    return shortest_paths_to

In [ ]:
# For each from_user, for each to_user, a list of shortest paths.
shortest_paths = {user.id: shortest_paths_from(user.id, friendships)
                  for user in users}

In [ ]:
shortest_paths

{0: {0: [[]],
  1: [[1]],
  2: [[2]],
  3: [[1, 3], [2, 3]],
  4: [[1, 3, 4], [2, 3, 4]],
  5: [[1, 3, 4, 5], [2, 3, 4, 5]],
  6: [[1, 3, 4, 5, 6], [2, 3, 4, 5, 6]],
  7: [[1, 3, 4, 5, 7], [2, 3, 4, 5, 7]],
  8: [[1, 3, 4, 5, 6, 8],
   [2, 3, 4, 5, 6, 8],
   [1, 3, 4, 5, 7, 8],
   [2, 3, 4, 5, 7, 8]],
  9: [[1, 3, 4, 5, 6, 8, 9],
   [2, 3, 4, 5, 6, 8, 9],
   [1, 3, 4, 5, 7, 8, 9],
   [2, 3, 4, 5, 7, 8, 9]]},
 1: {1: [[]],
  0: [[0]],
  2: [[2]],
  3: [[3]],
  4: [[3, 4]],
  5: [[3, 4, 5]],
  6: [[3, 4, 5, 6]],
  7: [[3, 4, 5, 7]],
  8: [[3, 4, 5, 6, 8], [3, 4, 5, 7, 8]],
  9: [[3, 4, 5, 6, 8, 9], [3, 4, 5, 7, 8, 9]]},
 2: {2: [[]],
  0: [[0]],
  1: [[1]],
  3: [[3]],
  4: [[3, 4]],
  5: [[3, 4, 5]],
  6: [[3, 4, 5, 6]],
  7: [[3, 4, 5, 7]],
  8: [[3, 4, 5, 6, 8], [3, 4, 5, 7, 8]],
  9: [[3, 4, 5, 6, 8, 9], [3, 4, 5, 7, 8, 9]]},
 3: {3: [[]],
  1: [[1]],
  2: [[2]],
  4: [[4]],
  0: [[1, 0], [2, 0]],
  5: [[4, 5]],
  6: [[4, 5, 6]],
  7: [[4, 5, 7]],
  8: [[4, 5, 6, 8], [4, 5, 7, 8]],
 

In [ ]:
# Now we can compute betweenness centrality

betweenness_centrality = {user.id: 0.0 for user in users}

for source in users:
    for target_id, paths in shortest_paths[source.id].items():
        if source.id < target_id:      # don't double count
            num_paths = len(paths)     # how many shortest paths?
            contrib = 1 / num_paths    # contribution to centrality
            for path in paths:
                for between_id in path:
                    if between_id not in [source.id, target_id]:
                        betweenness_centrality[between_id] += contrib

In [ ]:
betweenness_centrality

{0: 0.0,
 1: 3.5,
 2: 3.5,
 3: 18.0,
 4: 20.0,
 5: 20.5,
 6: 6.0,
 7: 6.0,
 8: 8.5,
 9: 0.0}

Users 0 and 9 have centrality 0 (as neither is on any shortest path between other users), whereas 3, 4, and 5 all have high centralities (as all three lie on many shortest paths).

<img src="images/fig01-03.png" alt="Analytics, Data Science, Machine Learning" style="width: 800px;"/>

Generally the centrality numbers aren’t that meaningful themselves. What we care about is how the numbers for each node compare to the numbers for other nodes.

### 3. Closeness Centrality
First, for each user we compute her `farness`, which is the sum of the lengths of her shortest paths to each other user. Since we’ve already computed the shortest paths between each pair of nodes, it’s easy to add their lengths. (If there are multiple shortest paths, they all have the same length, so we can just look at the first one.)

In [ ]:
def farness(user_id: int) -> float:
    """the sum of the lengths of the shortest paths to each other user"""
    return sum(len(paths[0])
               for paths in shortest_paths[user_id].values())

In [ ]:
# after which it’s very little work to compute closeness centrality
closeness_centrality = {user.id: 1 / farness(user.id) for user in users}

In [ ]:
closeness_centrality

{0: 0.029411764705882353,
 1: 0.037037037037037035,
 2: 0.037037037037037035,
 3: 0.045454545454545456,
 4: 0.05,
 5: 0.05,
 6: 0.041666666666666664,
 7: 0.041666666666666664,
 8: 0.03571428571428571,
 9: 0.027777777777777776}

<img src="images/fig01-04.png" alt="Analytics, Data Science, Machine Learning" style="width: 800px;"/>

There is much less variation here - even the very central nodes are still pretty far from the nodes out on the periphery.

Computing `shortest paths` is kind of a pain. For this reason, `betweenness and closeness centrality` aren’t often used on large networks. The less intuitive (but generally easier to compute) `eigenvector centrality` is more frequently used.

### 4. Eigenvector Centrality

## Eigenvector Centrality

Eigenvector centrality is a metric that assesses the influence or importance of a node within a network. Unlike degree centrality, which simply counts the number of connections a node has, eigenvector centrality considers the quality of those connections.

Essentially, a node's eigenvector centrality is high if it is connected to many nodes that also have high eigenvector centrality. This creates a recursive relationship where influential nodes are those connected to other influential nodes. It's a measure of indirect influence, capturing the idea that connections to important nodes are more valuable than connections to less important ones.

In the context of social networks, a person with high eigenvector centrality is likely to be a popular and influential figure, connected to many other influential individuals. In graph theory, it's mathematically defined as the principal eigenvector of the adjacency matrix of the network.

Eigenvector centrality has found applications in various fields, including social network analysis, information retrieval, and recommendation systems. It helps identify key players in networks, understand information flow, and make predictions about the behavior of the system.





In [ ]:
from scratch.linear_algebra import Matrix, make_matrix, shape

def matrix_times_matrix(m1: Matrix, m2: Matrix) -> Matrix:
    nr1, nc1 = shape(m1)
    nr2, nc2 = shape(m2)
    assert nc1 == nr2, "must have (# of columns in m1) == (# of rows in m2)"

    def entry_fn(i: int, j: int) -> float:
        """dot product of i-th row of m1 with j-th column of m2"""
        return sum(m1[i][k] * m2[k][j] for k in range(nc1))
    return make_matrix(nr1, nc2, entry_fn)

In [ ]:
from scratch.linear_algebra import Vector, dot

def matrix_times_vector(m: Matrix, v: Vector) -> Vector:
    nr, nc = shape(m)
    n = len(v)
    assert nc == n, "must have (# of cols in m) == (# of elements in v)"

    return [dot(row, v) for row in m]  # output has length nr

In [ ]:
from typing import Tuple
import random
from scratch.linear_algebra import magnitude, distance

def find_eigenvector(m: Matrix,
                     tolerance: float = 0.00001) -> Tuple[Vector, float]:
    guess = [random.random() for _ in m]

    while True:
        result = matrix_times_vector(m, guess)    # transform guess
        norm = magnitude(result)                  # compute norm
        next_guess = [x / norm for x in result]   # rescale

        if distance(guess, next_guess) < tolerance:
            # convergence so return (eigenvector, eigenvalue)
            return next_guess, norm
        guess = next_guess

In [ ]:
def entry_fn(i: int, j: int):
    return 1 if (i, j) in friend_pairs or (j, i) in friend_pairs else 0

n = len(users)
adjacency_matrix = make_matrix(n, n, entry_fn)

In [ ]:
eigenvector_centralities, _ = find_eigenvector(adjacency_matrix)

In [ ]:
eigenvector_centralities

[0.3857806077033853,
 0.5147894100194471,
 0.5147894100194471,
 0.4733139122016508,
 0.2336078915138231,
 0.15015018804392907,
 0.08355253092634635,
 0.08355253092634635,
 0.07284431114848253,
 0.02729321230231473]

<img src="images/fig01-05.png" alt="Analytics, Data Science, Machine Learning" style="width: 800px;"/>

Users with high `eigenvector centrality` should be those who have a lot of connections, and connections to people who themselves have high centrality.

Here users 1 and 2 are the most central, as they both have three connections to people who are themselves highly central. As we move away from them, people’s centralities steadily drop off.

On a network this small, eigenvector centrality behaves somewhat erratically. If you try adding or subtracting links, you’ll find that small changes in the network can dramatically change the centrality numbers. In a much larger network, this would not particularly be the case.

In other words, `eigenvector centralities are numbers, one per user, such that each user’s value is a constant multiple of the sum of his neighbors’ values`. In this case centrality means being connected to people who themselves are central. The more centrality you are directly connected to, the more central you are. This is of course a circular definition — `eigenvectors` are the way of breaking out of the circularity.

## Directed Graphs

### Number of endorsements

We’ll `track endorsements` (source, target) that no longer represent a reciprocal relationship, but rather that source endorses target.

<img src="images/fig01-06.png" alt="Analytics, Data Science, Machine Learning" style="width: 800px;"/>


In [ ]:
endorsements = [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2),
                (2, 1), (1, 3), (2, 3), (3, 4), (5, 4),
                (5, 6), (7, 5), (6, 8), (8, 7), (8, 9)]

In [ ]:
from collections import Counter

endorsement_counts = Counter(target for source, target in endorsements)
endorsement_counts

Counter({1: 2, 0: 2, 2: 2, 3: 2, 4: 2, 6: 1, 5: 1, 8: 1, 7: 1, 9: 1})

### PageRank

`“Number of endorsements” is an easy metric to game`. All you need to do is create phony accounts and have them endorse you. Or arrange with your friends to endorse each other. (As users 0, 1, and 2 seem to have done.)

`A better metric would take into account who endorses you`. Endorsements from people who have a lot of endorsements should somehow count more than endorsements from people with few endorsements. This is the essence of the **PageRank** algorithm, used by Google to rank websites based on which other websites link to them, which other websites link to those, and so on.

A simplified version of **PageRank** looks like this:

1. There is a total of 1.0 (or 100%) PageRank in the network.
2. Initially this PageRank is equally distributed among nodes.
3. At each step, a large fraction of each node’s PageRank is distributed evenly among its outgoing links.
4. At each step, the remainder of each node’s PageRank is distributed evenly among all nodes.

In [ ]:
import tqdm

def page_rank(users: List[User],
              endorsements: List[Tuple[int, int]],
              damping: float = 0.85,
              num_iters: int = 100) -> Dict[int, float]:
    # Compute how many people each person endorses
    outgoing_counts = Counter(target for source, target in endorsements)

    # Initially distribute PageRank evenly
    num_users = len(users)
    pr = {user.id : 1 / num_users for user in users}

    # Small fraction of PageRank that each node gets each iteration
    base_pr = (1 - damping) / num_users

    for iter in tqdm.trange(num_iters):
        next_pr = {user.id : base_pr for user in users}  # start with base_pr

        for source, target in endorsements:
            # Add damped fraction of source pr to target
            next_pr[target] += damping * pr[source] / outgoing_counts[source]

        pr = next_pr

    return pr

In [ ]:
# Compute page rank
pr = page_rank(users, endorsements)

# Thor (user_id 4) has higher page rank than anyone else
assert pr[4] > max(page_rank
                   for user_id, page_rank in pr.items()
                   if user_id != 4)

100%|██████████| 100/100 [00:00<00:00, 112237.20it/s]


In [ ]:
pr

{0: 0.1,
 1: 0.1,
 2: 0.1,
 3: 0.1,
 4: 0.14250000000000002,
 5: 0.1,
 6: 0.1,
 7: 0.1,
 8: 0.1,
 9: 0.1}

<img src="images/fig01-07.png" alt="Analytics, Data Science, Machine Learning" style="width: 800px;"/>

PageRank identifies user 4 (Thor) as the highest-ranked data scientist. Even though Thor has fewer endorsements (two) than users 0, 1, and 2, his endorsements carry with them rank from their endorsements. Additionally, both of his endorsers endorsed only him, which means that he doesn’t have to divide their rank with anyone else.

## Resources

- [NetworkX](https://networkx.github.io/) - NetworkX is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.
- [Gephi](https://gephi.org/) - Gephi is the leading visualization and exploration software for all kinds of graphs and networks. Gephi is open-source and free.